In [1]:
import requests, json, re, os
import xmltodict
import fastprogress
import zipfile
from pathlib import Path
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import fastprogress

In [8]:
# !curl -L https://github.com/narain1/dotfiles/releases/download/data/JsonData.zip -o json_data.zip
# !unzip -q json_data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   651  100   651    0     0  40687      0 --:--:-- --:--:-- --:--:-- 40687
100  138M  100  138M    0     0  11.2M      0  0:00:12  0:00:12 --:--:-- 13.3M


In [10]:
!curl -L https://github.com/narain1/dotfiles/releases/download/data/JsonData-2831.files.zip -o json_data_2831.zip
!unzip -q json_data_2831.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   662  100   662    0     0   2267      0 --:--:-- --:--:-- --:--:--  2267
100  302M  100  302M    0     0  5713k      0  0:00:54  0:00:54 --:--:-- 9637k
replace JsonData/ALDRBA_0X32E87E1615152D1A21B2EA40.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [12]:
!curl -L https://github.com/narain1/dotfiles/releases/download/data/JsonData-4874.files.zip -o json_data_4874.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   662  100   662    0     0   2081      0 --:--:-- --:--:-- --:--:--  2081
100  533M  100  533M    0     0  8173k      0  0:01:06  0:01:06 --:--:-- 7199k


In [2]:
json_path = 'JsonData'
fs = list(map(lambda y: os.path.join(json_path, y), filter(lambda x: x.endswith('.json'), os.listdir(json_path))))

In [3]:
required_fields = ['isComplete', 'isConflicted',
         'isCopyable', 'isDeletable', 'instType', 'parent',
        'isPart', 'type', 'number', 'description', 'name']

filter_items = ()

os.makedirs('filtered_jsons', exist_ok=True)

for k, file in enumerate(fastprogress.progress_bar(fs)):
    if os.path.isdir(file): continue
    with open(file, 'r') as f: o = json.load(f)
    acc_dict = []
    if o['MODEL']['ELECTRICAL']['CREATED'] == None: continue
    for item in o['MODEL']['ELECTRICAL']['CREATED']['INSTANCE']:
        d = dict()
        for r in required_fields:
            d[r] = item[f'{r}']
        d['CSTIC'] = list(filter(lambda x: x['isRequired'] == 'Y' or x['number'] in filter_items, item['CSTIC']))
        acc_dict.append(d)
    json_out = json.dumps({'content': acc_dict})
    with open(f'filtered_jsons/{Path(file).name}', 'w') as out_file:
        out_file.write(json_out)

In [4]:
filtered_files = list(map(lambda y: os.path.join('filtered_jsons', y), filter(lambda x: x.endswith('.json'), os.listdir('filtered_jsons'))))

In [5]:
acc = []
for file in fastprogress.progress_bar(filtered_files):
    with open(file, 'r') as f: o = json.load(f)
    for item in o['content']:
        acc.append(' '.join(list(map(lambda x: x['name'] + '_' + x['value'], item['CSTIC']))))

In [6]:
def process_json(file):
    with open(file, 'r') as f: o = json.load(f)
    dict_acc = dict()
    for item in o['content']:
        name = item['name']
        dict_acc[(item['parent'], item['number'])] = ' '.join(list(map(lambda x: name+ '__' + '_'.join(x['name'].split(' ')) + '__' + '_'.join(x['value'].split()), item['CSTIC'])))
    return dict_acc

In [7]:
corpus = []
for file in filtered_files:
    # with open(file, 'r') as f: json.load(f)
    o = process_json(file)
    for i in range(len(o)):
        if i == o: continue
        G = nx.DiGraph()
        G.add_edges_from(o.keys())
        corpus.append(' '.join(o[sent] for sent in list(filter(lambda x: x[1] in nx.shortest_path(G, '1', str(i+1)), o.keys()))))

In [8]:
tfidf = TfidfVectorizer()
query_matrix = tfidf.fit_transform(corpus)

In [9]:
query_matrix

<38488x5654 sparse matrix of type '<class 'numpy.float64'>'
	with 2584652 stored elements in Compressed Sparse Row format>

In [10]:
classes = []
for file in fastprogress.progress_bar(fs):
    if os.path.isdir(file): continue
    with open(file, 'r') as f: o = json.load(f)
    if o['MODEL']['ELECTRICAL']['CREATED'] is None: continue
    for item in o['MODEL']['ELECTRICAL']['CREATED']['INSTANCE']:
        if item['name'] not in classes: classes.append(item['name'])

In [24]:
# from fastprogress.fastprogress import master_bar, progress_bar

# acc_dict = dict()
# parent_bar = master_bar(classes)
# for c in parent_bar:
#     d = dict()
#     for file in progress_bar(fs, parent=parent_bar):
#         if os.path.isdir(file): continue
#         with open(file, 'r') as f: o = json.load(f)
#         if o['MODEL']['ELECTRICAL']['CREATED'] is None: continue
#         for item in o['MODEL']['ELECTRICAL']['CREATED']['INSTANCE']:
#             if item['name'] == c:
#                 for item1 in item['CSTIC']:
#                     if item1['name'] not in d: 
#                         d[item1['name']] = list(set(map(lambda x: '_'.join(x.lower().split()), item1['options'].split('|'))))
#     acc_dict[c] = d

In [12]:
acc_dict = dict()

for file in fastprogress.progress_bar(fs):
    with open(file, 'r') as f: o = json.load(f)
    if o['MODEL']['ELECTRICAL']['CREATED']['INSTANCE'] is None: continue
    for item in o['MODEL']['ELECTRICAL']['CREATED']['INSTANCE']:
        name = item['name']
        for item1 in item['CSTIC']:
            if item1['isRequired'] == 'N': continue
            if name in acc_dict:
                if item1['name'] in acc_dict[name]:
                    v = '_'.join(item1['value'].split()).lower()
                    if v not in acc_dict[name][item1['name']] and item1['value'] != '': acc_dict[name][item1['name']].append(v)
                else: acc_dict[name][item1['name']] = []
            else:
                acc_dict[name] = {item1['name']: [item1['value']]}

In [14]:
# acc_dict

In [60]:
def generate_combinations(o1):
    acc = []
    for k, v in o1.items():
        dict_acc = []
        for item in acc_dict[k[0]][k[1]]:
            if k[1] == item: continue
            d = dict()
            for k1, v1 in o1.items():
                d[(k1[0], k1[1], item if k1 == k else k1[2])] = v1
            dict_acc.append(d)
        acc.extend(dict_acc)
    return acc

make_sentence = lambda x: ' '.join(reduce(lambda x,y: x+y, [[i[-1]]*j for i,j in x.items()]))

In [61]:
def generate_combinations(o):
    G = nx.DiGraph()
    G.add_edges_from(list(o.keys()))
    d_acc = dict()
    for n in o:
        if n[0] == '-1': continue
        path = nx.shortest_path(G, '-1', n[1])
        path.remove(n[1])
        parent_sents = [o[ll] for ll in o.keys() if ll[1] in path]
        items = o[n].split()
        d_acc_1 = [' '.join([*parent_sents, o[n]])]
        for i in items:
            temp = i.split('__')
            for i1 in acc_dict[temp[0]][temp[1]]:
                sent_acc = []
                for i3 in items:
                    if i3 == i: sent_acc.append('__'.join([*i.split('__')[:-1], i1]))
                    else: sent_acc.append(i3)
                d_acc_1.append('  '.join(parent_sents + sent_acc))
        d_acc[n] = d_acc_1
    return d_acc

In [62]:
def generate_combinations_3(o):
    G = nx.DiGraph()
    G.add_edges_from(list(o.keys()))
    sents = []
    for k, v in o.items():
        sent = [f'{k[0]}_{k[1]}__{v1}' for v1 in v.split()]
        sents.append(' '.join(sent))
        
    acc = [' '.join(sents)]
    for o1 in ' '.join(sents).split():
        acc1 = []
        a1 = o1.split('__')
        for o2 in acc_dict[a1[1]][a1[2]]:
            # if a1[-1].lower() == o2.lower(): continue
            if a1[-1].lower() == '_'.join(o2.split()).lower(): continue
            # if re.match('^[0-9]*$', o2) and eval(o2) == eval(a1[-1]): continue
            comb = []
            for o3 in ' '.join(sents).split():
                if o1.lower() == o3.lower(): comb.append('__'.join(o3.split('__')[:-1] + [o2]))
                else: comb.append(o3)
            acc.append(' '.join(comb))
    return acc

In [112]:
o = process_json(filtered_files[5])
out = generate_combinations_3(o)

In [113]:
preprocess_sentence = lambda y: ' '.join(map(lambda x: '__'.join(x.split('__')[1:]), y.split()))

In [114]:
sims = cosine_similarity(query_matrix, tfidf.transform(map(preprocess_sentence, out))).max(axis=0)

In [100]:
def format_out(o):
    d = dict()
    for i in o.split():
        a = i.split('__')
        node = tuple(a[0].split('_'))
        if node in d: d[node].append(tuple(a[1:]))
        else: d[node] = [tuple(a[1:])]
    return d

def flatten_sentence(o):
    sents = []
    for k, v in o.items():
        sent = [f'{k[0]}_{k[1]}__{v1}' for v1 in v.split()]
        sents.append(' '.join(sent))
    return ' '.join(sents)

In [115]:
k = 5
idx = np.argpartition(sims[1:], -k)[-k:]

In [116]:
ooo = [out[i+1] for i in idx]

In [118]:
for i in range(5):
    print(i, format_out(' '.join(list(set(flatten_sentence(o).split()).symmetric_difference(ooo[i].split())))))

0 {('2', '6'): [('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(1)#14-#10_CU_/_#12-#10_AL'), ('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(2)3/0-250kcmil_cu/al')]}
1 {('2', '8'): [('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(3)1/0-500kcmil'), ('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(1)#14-#10_CU_/_#12-#10_AL')]}
2 {('2', '8'): [('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(1)#3-300kcmil_cu/al'), ('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(1)#14-#10_CU_/_#12-#10_AL')]}
3 {('2', '6'): [('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(1)#14-#10_CU_/_#12-#10_AL'), ('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(3)1/0-500kcmil')]}
4 {('2', '6'): [('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(1)#3-300kcmil_cu/al'), ('TYZ:PB_VL_MCCB', 'A7JPB_LUG_WIRE_RANGE', '(1)#14-#10_CU_/_#12-#10_AL')]}


In [125]:
def get_recomendations(file, k=5):
    o = process_json(file)
    out = generate_combinations_3(o)
    preprocess_sentence = lambda y: ' '.join(map(lambda x: '__'.join(x.split('__')[1:]), y.split()))
    sims = cosine_similarity(query_matrix, tfidf.transform(map(preprocess_sentence, out))).max(axis=0)
    idx = np.argpartition(sims[1:], -k)[-k:]
    ooo = [out[i+1] for i in idx]
    return [format_out(' '.join(list(set(flatten_sentence(o).split()).symmetric_difference(ooo[i].split())))) for i in range(k)]

In [127]:
filtered_files[0]


'filtered_jsons/TANKGX00C_0X411F7E2841E15E17.json'

In [132]:
get_recomendations(filtered_files[0], k=10)

[{('-1', '1'): [('TYZ:PANELBOARD', 'A7J_DESCRIPTION', 'panelboard_p2'),
   ('TYZ:PANELBOARD', 'A7J_DESCRIPTION', 'PANELBOARD_P1')]},
 {('-1', '1'): [('TYZ:PANELBOARD', 'A7JPB_ENGRAV_SIZE_MASTER_NP', '3/8'),
   ('TYZ:PANELBOARD', 'A7JPB_ENGRAV_SIZE_MASTER_NP', '3/16')]},
 {('-1', '1'): [('TYZ:PANELBOARD', 'A7JPB_ENCLOSURE_FRONT', 'special'),
   ('TYZ:PANELBOARD', 'A7JPB_ENCLOSURE_FRONT', 'STANDARD')]},
 {('-1', '1'): [('TYZ:PANELBOARD', 'A7JPB_BUS_MATERIAL', 'copper'),
   ('TYZ:PANELBOARD', 'A7JPB_BUS_MATERIAL', 'ALUM')]},
 {('-1', '1'): [('TYZ:PANELBOARD', 'A7JPB_FEED', 'bottom'),
   ('TYZ:PANELBOARD', 'A7JPB_FEED', 'TOP')]},
 {('-1', '1'): [('TYZ:PANELBOARD', 'A7JPB_SEISMICALLY_QUALIFIED', 'TRUE'),
   ('TYZ:PANELBOARD', 'A7JPB_SEISMICALLY_QUALIFIED', 'false')]},
 {('1', '2'): [('TYZ:PB_MAINS', 'A7JPB_FEED', 'TOP'),
   ('TYZ:PB_MAINS', 'A7JPB_FEED', 'bottom')]},
 {('1', '2'): [('TYZ:PB_MAINS', 'A7JPB_LUG_WIRE_RANGE', '(1)#12-1/0'),
   ('TYZ:PB_MAINS', 'A7JPB_LUG_WIRE_RANGE', '(1)#6-350

In [124]:
filtered_files[5]

'filtered_jsons/SCOTJX20C_0X77D37E3711101536.json'

In [129]:
len(acc_dict)

72

In [130]:
acc_dict.keys()

dict_keys(['TYZ:PANELBOARD', 'TYZ:PB_MAINS', 'TYZ:PB_VL_MCCB', 'TYZ:PB_VL', 'TYZ:PB_BL', 'TYZ:PB_CUST_MTR', 'TYZ:PB_DIGT_MTR', 'TYZ:PB_FD', 'TYZ:PB_SPCL_MOD', 'TYZ:PB_JD', 'TYZ:PB_3V_3VAU', 'TYZ:PB_SPD_TPS3', 'TYZ:PB_QJ', 'TYZ:PB_ED', 'TYZ:PB_LD', 'TYZ:PB_TIMECLOCK', 'TYZ:PB_SLD', 'TYZ:PB_QP', 'TYZ:PB_SND', 'TYZ:PB_BFDR_CNT', 'TYZ:PB_RCS', 'TYZ:PB_BFDR_VBS', 'TYZ:PB_BFDR_HCP', 'TYZ:PB_SJD', 'TYZ:SB_SWITCHBOARD', 'TYZ:SB_INCOMING', 'TYZ:SB_UTIL_MTR', 'TYZ:SB_VLMCCB', 'TYZ:SB_FD', 'TYZ:SB_VL', 'TYZ:SB_BQD', 'TYZ:SB_FD_PNL_BD', 'TYZ:SB_FACT_OPT', 'TYZ:MS_MCC', 'TYZ:MC_MCB', 'TYZ:MC_SPECIAL', 'TYZ:MC_PILOTLIT', 'TYZ:MC_PLC', 'TYZ:MC_ANG_METR', 'TYZ:MC_PBD', 'TYZ:MC_FCB', 'TYZ:MC_ED_PO', 'TYZ:MC_LD_PO', 'TYZ:MCC', 'TYZ:MC_INC_MLO', 'TYZ:MC_FCB_DL', 'TYZ:MC_LEFT_FCB', 'TYZ:MC_RIGT_FCB', 'TYZ:MC_FVNR', 'TYZ:MC_FD_PO', 'TYZ:MC_METERUNI', 'TYZ:MC_DIG_METR', 'TYZ:MC_VFD', 'TYZ:MC_DLFDXFMR', 'TYZ:MC_DL_XFMR', 'TYZ:MC_XFMR_UNI', 'TYZ:PB_BQD', 'TYZ:SB_FDR_HCP', 'TYZ:SB_HCP', 'TYZ:SB_RD', 'TYZ:PB_MD

In [131]:
query_matrix

<38488x5654 sparse matrix of type '<class 'numpy.float64'>'
	with 2584652 stored elements in Compressed Sparse Row format>